
## Wedge Task 2
This file connects to my GBQ database, builds a list of owners and takes a sample of those owner. We then export a csv file.  

In [37]:
# Do our imports for the code
from google.cloud import bigquery
from google.oauth2 import service_account
import random #for random sample
import pyarrow

In [38]:
#Here I connect to my GBQ . Need to always verify I'm connecting to print project and dataset!

# These first two values will be different on your machine. 
service_path = "c:\\Users\\logan\\DataScience\\"
service_file = 'beskoonlettuce-327919-af8cb2931ac8.json' #change this to your authentication information  
gbq_proj_id = 'beskoonlettuce-327919' # change this to your poroject. 
#gbq_dataset_id = 'wedge_transactions' # and change this to your data set ID

# And this should stay the same. 
private_key =service_path + service_file

In [39]:
# Now we pass in our credentials so that Python has permission to access our project.
credentials = service_account.Credentials.from_service_account_file(service_path + service_file)

In [40]:
# And finally we establish our connection
client = bigquery.Client(credentials = credentials, project=gbq_proj_id) 
#client = bigquery.Client()

In [41]:
# Set up query to pull list of all the owners from transactions.
query = (
    "Select DISTINCT card_no,"
    "FROM `beskoonlettuce-327919.wedge_transactions.transArchive_*`"
    "WHERE card_no != 3 "
    #"GROUP BY card_no "
)

#Execute query with `client.query`
query_job = client.query(
    query,
)

In [45]:
#owners.clear()
#print(owners)
df1 = client.query(query).to_dataframe()

In [46]:
display(df1)

,card_no
0,47333.0
1,47389.0
2,47943.0
3,48022.0
4,48438.0
...,...
27202,48694.0
27203,48712.0
27204,48594.0
27205,52300.0


In [49]:
# Here I generate the random sample of owners

sample_owners = df1.sample(n=1000)
display(sample_owners)

#If I wanted to put the samples in an array.
#arr = sample_owners.to_numpy()
#print(arr)

,card_no
4927,49488.0
24031,51131.0
2045,27016.0
10766,64866.0
7427,35900.0
...,...
10422,49967.0
12465,42966.0
26531,25870.0
15561,37452.0


In [50]:
samples = sample_owners.values.tolist()
samples1 = [i[0] for i in samples]# this lil fella, he saves the day. He makes it a perfect list to query.
print(samples1)

[49488.0, 51131.0, 27016.0, 64866.0, 35900.0, 25036.0, 21847.0, 51030.0, 14227.0, 22537.0, 19768.0, 26862.0, 10609.0, 20903.0, 31021.0, 23251.0, 19044.0, 24820.0, 52051.0, 18748.0, 19016.0, 51221.0, 14619.0, 16537.0, 36087.0, 65806.0, 64361.0, 37521.0, 21523.0, 18022.0, 10299.0, 64403.0, 49428.0, 17796.0, 12106.0, 17020.0, 10899.0, 51950.0, 11606.0, 17207.0, 64687.0, 17060.0, 15873.0, 15246.0, 11822.0, 15605.0, 44519.0, 18316.0, 10755.0, 17822.0, 24101.0, 12845.0, 36842.0, 11391.0, 34624.0, 25700.0, 46638.0, 16814.0, 64552.0, 51495.0, 13599.0, 27608.0, 14797.0, 52586.0, 52256.0, 24779.0, 51247.0, 52486.0, 25205.0, 64851.0, 16434.0, 16139.0, 24608.0, 21230.0, 23831.0, 42778.0, 48420.0, 52590.0, 17043.0, 52205.0, 23230.0, 40588.0, 65472.0, 55590.0, 49058.0, 16741.0, 22628.0, 18014.0, 19460.0, 52767.0, 49705.0, 52915.0, 13847.0, 11629.0, 14256.0, 58172.0, 21428.0, 46734.0, 51721.0, 24527.0, 19301.0, 47819.0, 24248.0, 21110.0, 17979.0, 25473.0, 54663.0, 17641.0, 11922.0, 34777.0, 13140.0, 

In [51]:
# Query to pull sample owners data from GBQ
samplest =tuple(samples1)

query1 = (
    "Select *"
    "FROM `beskoonlettuce-327919.wedge_transactions.transArchive_*`"
    "where card_no IN {};".format(samplest)
)
#print(query1)
#job_config = bigquery.QueryJobConfig(
 #   query_parameters=[
 #   bigquery.ArrayQueryParameter('samples', 'type':'STRING', samples)]
#)
# And we execute queries with `client.query`
query_job = client.query(
    query1,
    #job_config=job_config,
    location="US",
)

In [52]:
df = client.query(query1).to_dataframe()

In [53]:
#Once the cell above is complete with the query (may take awhile), we can read the records in the data.
df.head()

,datetime,register_no,emp_no,trans_no,upc,description,trans_type,trans_subtype,trans_status,department,...,batchHeaderID,local,organic,display,receipt,card_no,store,branch,match_id,trans_id
0,2013-10-26 16:24:24+00:00,2.0,72.0,8.0,0,InStoreCoupon,T,IC,,0.0,...,None,0.0,None,None,0.0,44300.0,1.0,0.0,0.0,46.0
1,2013-10-31 12:32:42+00:00,16.0,54.0,107.0,0,Check,T,CK,,0.0,...,None,0.0,None,None,0.0,44242.0,1.0,0.0,0.0,5.0
2,2013-10-31 12:49:01+00:00,5.0,71.0,10.0,0,Check,T,CK,,0.0,...,None,0.0,None,None,0.0,44242.0,1.0,0.0,0.0,19.0
3,2013-11-04 18:28:10+00:00,4.0,60.0,30.0,0,InStoreCoupon,T,IC,,0.0,...,None,0.0,None,None,0.0,47850.0,1.0,0.0,0.0,60.0
4,2013-11-06 11:49:17+00:00,7.0,60.0,53.0,0,EBT FS,T,EF,,0.0,...,None,0.0,None,None,0.0,48420.0,1.0,0.0,0.0,21.0


In [54]:
# The very last step is to write our dataframe to a csv file for local work.
df.to_csv('wedge_sample.csv', index=False)